In [20]:
# Importamos los paquetes usuales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
# Leer los archivos p y w
df_p = pd.read_csv('p_data.csv', delimiter = ";")
df_w = pd.read_csv('w_data.csv', delimiter = ";")

In [11]:
# Análisis exploratorio básico
df_p.head()


,Age,Sex,ID,Race,Country,Education,Education-Num,Marital Status,Relationship
0,39.0,Male,49084,White,United-States,Bachelors,13.0,Never-married,Not-in-family
1,50.0,Male,68553,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband
2,38.0,Male,143654,White,United-States,HS-grad,9.0,Divorced,Not-in-family
3,53.0,Male,41510,Black,United-States,11th,7.0,Married-civ-spouse,Husband
4,28.0,Female,195025,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife


In [12]:
df_w.head()

,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,ID,Target
0,State-gov,77516.0,2174.0,0.0,40.0,49084,<=50K
1,Self-emp-not-inc,83311.0,0.0,0.0,13.0,68553,<=50K
2,Private,215646.0,0.0,0.0,40.0,143654,<=50K
3,Private,234721.0,0.0,0.0,40.0,41510,<=50K
4,Private,338409.0,0.0,0.0,40.0,195025,<=50K


# Observamos que hay 16 columnas de datos en total


In [34]:
# Ordenamos el dataFrame df_p
df_p_ordered = df_p.sort_values('ID')
df_p_ordered.drop_duplicates(inplace = True)
print(df_p_ordered.shape)

# Ordenamos el dataFrame df_q
df_w_ordered = df_w.sort_values('ID')
df_w_ordered.drop_duplicates(inplace = True)
print(df_w_ordered.shape)

(48842, 9)
(48843, 7)


In [29]:
# Hacemos una serie con los duplicados
serie_duplicados_df_p = df_p['ID'].duplicated()
serie_duplicados_df_w = df_w['ID'].duplicated()
print(df_p[serie_duplicados_df_p].shape)
print(df_w[serie_duplicados_df_w].shape)

(0, 9)
(0, 7)
